# Deep Neural Network for MNIST Classification

The dataset is called MNIST and refers to handwritten digit recognition. You can find more about it on Yann LeCun's website (Director of AI Research, Facebook). He is one of the pioneers of what we've been talking about and of more complex approaches that are widely used today, such as covolutional neural networks (CNNs). 

The dataset provides 70,000 images (28x28 pixels) of handwritten digits (1 digit per image). 

The goal is to write an algorithm that detects which digit is written. Since there are only 10 digits (0, 1, 2, 3, 4, 5, 6, 7, 8, 9), this is a classification problem with 10 classes. 

Our goal would be to build a neural network with 2 hidden layers.

### Import relevant libraries

In [1]:
import numpy as np
import tensorflow as tf

# TensorFLow includes a data provider for MNIST that we'll use.

from tensorflow.keras.datasets import mnist

## Data

Load and preprocess the data.

In [8]:
#images dataset are 28x28 array that contain grayscale values 0 to 255, 
# labels contain the image label (e.g. 0, 1, 2, 3...)
(train_images,train_labels),(test_images,test_labels) = mnist.load_data(
                "G:/Meu Drive/PersonalProj/GitHub/Python_Statistics/Python_Deep_Learning/Data/mnist.npz")

# Extract the training and testing dataset with the built references

#Create Tensorflow Dataset
mnist_train = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
mnist_test = tf.data.Dataset.from_tensor_slices((test_images, test_labels))

# by default, TF has training and testing datasets, but no validation sets
# thus we must split it on our own

# we start by defining the number of validation samples as a % of the train samples
num_validation_samples = 0.1 * len(mnist_train)
# let's cast this number to an integer, as a float may cause an error along the way
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

# let's also store the number of test samples in a dedicated variable 
# (instead of using the mnist_info one)
num_test_samples = len(mnist_test)
# once more, we'd prefer an integer (rather than the default float)
num_test_samples = tf.cast(num_test_samples, tf.int64)


# Normally, we would scale our data in some way to make the result 
# more numerically stable.
# In this case we will simply prefer to have inputs between 0 and 1 
# Let's define a function called: scale, that will take an MNIST image and its label
def scale(image, label):
    # we make sure the value is a float
    image = tf.cast(image, tf.float32)
    # since the possible values for the inputs are 0 to 255 (256 different shades of grey)
    # if we divide each element by 255, we would get the desired result -> 
    # all elements will be between 0 and 1 
    image /= 255.

    return image, label


# the method .map() allows us to apply a custom transformation to a given dataset
# we have already decided that we will get the validation data from mnist_train, so 
scaled_train_and_validation_data = mnist_train.map(scale)

# finally, we scale and batch the test data
# we scale it so it has the same magnitude as the train and validation
# there is no need to shuffle it, because we won't be training on the test data
# there would be a single batch, equal to the size of the test data
test_data = mnist_test.map(scale)


# let's also shuffle the data

BUFFER_SIZE = 10000
# this BUFFER_SIZE parameter is here for cases when we're dealing with enormous datasets
# then we can't shuffle the whole dataset in one go because we can't fit it all in memory
# so instead TF only stores BUFFER_SIZE samples in memory at a time and shuffles them
# if BUFFER_SIZE=1 => no shuffling will actually happen
# if BUFFER_SIZE >= num samples => shuffling is uniform
# BUFFER_SIZE in between - a computational optimization to approximate uniform shuffling

# Use shuffle method readily available, just need to specify the buffer size
shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

# once we have scaled and shuffled the data, proceed to extract the train and validation
# our validation data would be equal to 10% of the training set, which we've already calculated
# we use the .take() method to take that many samples
# finally, we create a batch with a batch size equal to the total number of validation samples
validation_data = shuffled_train_and_validation_data.take(num_validation_samples)

# similarly, the train_data is everything else. Skip as many samples as there are in the 
# validation dataset
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

# determine the batch size
BATCH_SIZE = 100

# we can also take advantage of the occasion to batch the train data
# this would be very helpful when we train, as we would be able to iterate 
# over the different batches
train_data = train_data.batch(BATCH_SIZE)

validation_data = validation_data.batch(num_validation_samples)

# batch the test data
test_data = test_data.batch(num_test_samples)


# takes next batch (it is the only batch)
# because as_supervized=True, we've got a 2-tuple structure
validation_inputs, validation_targets = next(iter(validation_data))

## Model
### Outline the model
When thinking about a deep learning algorithm, we mostly imagine building the model. So, let's do it :)

In [9]:
input_size = 784
output_size = 10
# Use same hidden layer size for both hidden layers. Not a necessity.
hidden_layer_size = 50
    
# define how the model will look like
model = tf.keras.Sequential([
    
    # the first layer (the input layer)
    # each observation is 28x28x1 pixels, therefore it is a tensor of rank 3
    # since we don't know CNNs yet, we don't know how to feed such input into 
    # our net, so we must flatten the images there is a convenient method 
    # 'Flatten' that simply takes our 28x28x1 tensor and orders it into a (None,) 
    # or (28x28x1,) = (784,) vector
    # this allows us to actually create a feed forward neural network
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)), # input layer
    
    # tf.keras.layers.Dense is basically implementing: 
    # output = activation(dot(input, weight) + bias)
    # it takes several arguments, but the most important ones for us are 
    # the hidden_layer_size and the activation function
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 1st hidden layer
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 2nd hidden layer
    
    # the final layer is no different, we just make sure to activate it with softmax
    tf.keras.layers.Dense(output_size, activation='softmax') # output layer
])

### Choose the optimizer and the loss function

In [10]:
# we define the optimizer we'd like to use, the loss function, 
# and the metrics we are interested in obtaining at each iteration
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Training
That's where we train the model we have built.

In [11]:
# determine the maximum number of epochs
NUM_EPOCHS = 6

# we fit the model, specifying the training data
# the total number of epochs
# and the validation data we just created ourselves in the format: (inputs,targets)
model.fit(train_data, 
          epochs=NUM_EPOCHS, 
          validation_data=(validation_inputs, validation_targets), 
          verbose =2
          )

Epoch 1/6
540/540 - 6s - 11ms/step - accuracy: 0.8845 - loss: 0.4239 - val_accuracy: 0.9428 - val_loss: 0.2077
Epoch 2/6
540/540 - 3s - 5ms/step - accuracy: 0.9476 - loss: 0.1818 - val_accuracy: 0.9512 - val_loss: 0.1683
Epoch 3/6
540/540 - 3s - 6ms/step - accuracy: 0.9592 - loss: 0.1391 - val_accuracy: 0.9625 - val_loss: 0.1323
Epoch 4/6
540/540 - 3s - 6ms/step - accuracy: 0.9655 - loss: 0.1170 - val_accuracy: 0.9662 - val_loss: 0.1175
Epoch 5/6
540/540 - 3s - 6ms/step - accuracy: 0.9699 - loss: 0.1021 - val_accuracy: 0.9715 - val_loss: 0.0982
Epoch 6/6
540/540 - 3s - 6ms/step - accuracy: 0.9738 - loss: 0.0874 - val_accuracy: 0.9712 - val_loss: 0.1000


In [12]:
test_loss, test_accuracy = model.evaluate(test_data)

# We can apply some nice formatting if we want to
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 239ms/step - accuracy: 0.9683 - loss: 0.1076
Test loss: 0.11. Test accuracy: 96.83%
